In [1]:
#Load final training df as generated from notebook 1.4
import pandas as pd

train_df = pd.read_csv("train_df.csv") #assume df within repository and not in local data repositories
valid_df = pd.read_csv("valid_df.csv")

#todo name cols accordingly
train_df.head()

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,0.181925,0.016456,2.977172,0.251361,0.170225,1.177835,0.068797,0.300512,-48.956365,0.287949,...,0.005671,0.004881,0.006147,0.004177,0.005132,0.008367,0.008827,0.004732,0.030978,healthy
1,0.196951,0.012408,2.906090,0.271829,0.207477,1.272237,0.062872,0.322262,-41.986142,0.273570,...,0.006226,0.005230,0.010857,0.004436,0.006118,0.008744,0.009558,0.005070,0.029567,healthy
2,0.197094,0.012408,2.906155,0.284378,0.227773,1.273140,0.054850,0.322262,-41.190031,0.260472,...,0.006226,0.005230,0.010851,0.004436,0.006119,0.008744,0.009558,0.005071,0.029562,healthy
3,0.117633,0.000808,2.997585,0.385375,0.275210,2.650451,0.004746,0.637786,-35.052459,0.890239,...,0.012873,0.008890,0.022404,0.003368,0.007685,0.012432,0.013955,0.006946,0.009270,healthy
4,0.359369,0.000808,2.998085,0.385440,0.275234,2.650791,0.004746,0.637787,-35.052027,0.889768,...,0.012873,0.008890,0.022396,0.003355,0.007690,0.012434,0.013945,0.006954,0.009268,healthy


In [2]:
#convert df to input output X Y 
from sklearn.model_selection import train_test_split 
from sklearn import svm
#train set augmented
X_train = train_df.iloc[: , :-1] #everything except last column
y_train = train_df.iloc[:, 34] #last column
#test set not augmented 
X_test = valid_df.iloc[: , :-1]
y_test = valid_df.iloc[:, 34]



In [3]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

#normalization
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

#apply pca to reduce feature space 
pca = PCA(n_components=0.95) #0.95 variance
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.fit_transform(X_test_scaled)

print(f"Number of components chosen: {pca.n_components_}")

Number of components chosen: 23


In [4]:
import pandas as pd
from sklearn import svm
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report


# 3. Define the Complex Model Search
# RBF kernel handles non-linear patterns by mapping to infinite dimensions
param_grid = {
    'C': [0.1, 1, 10, 100, 1000],          # Penalty for misclassification
    'gamma': [1, 0.1, 0.01, 0.001],        # Kernel coefficient (influence radius)
    'kernel': ['rbf']                      # The 'complex' kernel
}

# 4. Use GridSearchCV to find the best version of the model
grid = GridSearchCV(svm.SVC(), param_grid, refit=True, verbose=2, cv=5)
grid.fit(X_train_pca, y_train)


Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   3.7s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   3.6s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   3.7s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   3.6s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   3.8s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=   3.9s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=   3.7s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=   3.9s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=   3.7s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=   3.7s
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time=   3.7s
[CV] END ......................C=0.1, gamma=0.0

,estimator,SVC()
,param_grid,"{'C': [0.1, 1, ...], 'gamma': [1, 0.1, ...], 'kernel': ['rbf']}"
,scoring,None
,n_jobs,None
,refit,True
,cv,5
,verbose,2
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,C,0.1


In [5]:
# 5. Final Model
best_model = grid.best_estimator_
y_predict = best_model.predict(X_test_pca)

In [6]:
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

print("\nClassification Report:")
print(classification_report(y_test, y_predict))


print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_predict))


Classification Report:
              precision    recall  f1-score   support

    COVID-19       0.07      0.31      0.11       263
     healthy       0.76      0.44      0.56      3095
 symptomatic       0.20      0.30      0.24       775

    accuracy                           0.40      4133
   macro avg       0.34      0.35      0.30      4133
weighted avg       0.61      0.40      0.47      4133


Confusion Matrix:
[[  81   97   85]
 [ 886 1360  849]
 [ 218  325  232]]
